In [ ]:
# noexport

import os
os.system('export_notebook browser_libs.ipynb')

In [44]:
import json
import urllib.request as req
from memoize import memoize # pip install memoize2
from pymongo import MongoClient
from getsecret import getsecret
import urllib.parse
import moment

In [ ]:
#def get_user_to_all_install_ids():
#  user_to_install = json.loads(req.urlopen("http://localhost:5001/get_user_to_all_install_ids").read().decode("utf-8"))
#  return user_to_install

#def get_collection_names():
#  collection_names = json.loads(req.urlopen("http://localhost:5001/listcollections").read().decode("utf-8"))
#  return collection_names

#def get_session_info_list_for_user(userid):
#  output = json.loads(req.urlopen("http://localhost:5001/get_session_info_list_for_user?userid=" + userid).read().decode("utf-8"))
#  return output

In [ ]:
#collection_names = get_collection_items('collections')
#print(len(collection_names))
#print(get_collection_for_user('e0ea34c81d4b50cddc7bd752', 'synced:seconds_on_domain_per_session')[0])

In [13]:
@memoize
def download_url(url):
  return req.urlopen(url).read().decode("utf-8")

def getjson(path, params={}):
  querystring = urllib.parse.urlencode(params)
  url = 'http://localhost:5001/' + path + '?' + querystring
  return json.loads(download_url(url))

def make_getjson_func(path, *param_list):
  def f(*arg_list):
    if len(param_list) != len(arg_list):
      print('missing some number of arguments. expected parameters: ' + str(param_list))
    param_dict = {}
    for param,arg in zip(param_list, arg_list):
      param_dict[param] = arg
    return getjson(path, param_dict)
  return f

def expose_getjson(func_name, *args):
  f = make_getjson_func(func_name, *args)
  globals()[func_name] = f
  return f  


In [15]:
expose_getjson('get_session_info_list_for_user', 'userid')
#expose_getjson('get_user_to_all_install_ids')

#print(get_user_to_all_install_ids()['e0ea34c81d4b50cddc7bd752'])
#get_session_info_list = make_getjson_func('get_session_info_list_for_user', 'userid')
#print(get_session_info_list_for_user('e0ea34c81d4b50cddc7bd752')[0])
#def get_user_to_all_install_ids(user):
#  return getjson

['803aa33d5eaf0c561e8a3728', '06b5c1a6b60318169aa70015', '825864bc8702afff28aacbbc', 'c19500c7b139be66b502d51f', '9ef2f94629377eefe13720d8', 'f975c9c7e528003255dbe143', 'a37c1102fa973cde1fc1f218', 'd9e261705e7b6255dad42c05', '4b98d3dc38e634d62a53f2ed', '98f84f820b8a32a14c765b29', 'b6473a43e542ef940d31bc41', '61b72b717a78c83faf858b8a', 'cc99d1226da6a733898a3c42', 'ed1cd8ffdb099cb63de09d12', '11827b1959b8af192ab32d5c', '4034c1f5256318dc17ea4ee5', '03fb12d03bdc27485945f5a0', 'ebc3601374640468c4ee064d', '7b3f19bc5cf18b977643fb2a', 'e4953386e2ab82c992dd8c98', '019abdd547d520d09e60d410', '9af88d32b09871118441897b', '7a994ccad67df2369e53541a', '1af2e4e2e83a33943139f37a', 'f1e6659ea775d3f2e016df24', 'dd858122bc39012780233320', '07560b61241dba2e7d75263c', '795ea44c58a60dd84e889801', '405aaa6411d349079fc85496', '13e0f14fbbcffb031c2c5bc4']


In [12]:
@memoize
def get_db(): # this is for the browser
  client = MongoClient(getsecret("EXT_URI"))
  db = client[getsecret("DB_NAME")]
  return db

@memoize
def get_collection_items(collection_name):
  db = get_db()
  return [x for x in db[collection_name].find({})]

def get_collection_for_user(user, collection_name):
  return get_collection_items(user + '_' + collection_name)


In [19]:
def get_collection_names():
  collection_names = get_collection_items('collections')
  return [x['_id'] for x in collection_names]

def get_users_with_goal_frequency_set():
  output = []
  collection_names = get_collection_names()
  for collection_name in collection_names:
    if not collection_name.endswith('_synced:goal_frequencies'):
      continue
    username = collection_name.replace('_synced:goal_frequencies', '')
    output.append(username)
  return output


In [41]:
@memoize
def get_user_to_all_install_ids():
  install_info_list = get_collection_items('installs')
  output = {}
  for install_info in install_info_list:
    if 'user_id' not in install_info:
      continue
    user_id = install_info['user_id']
    install_id = install_info.get('install_id', None)
    if user_id not in output:
      output[user_id] = []
    if install_id not in output[user_id]:
      output[user_id].append(install_id)
  return output

#print(get_user_to_all_install_ids()['e0ea34c81d4b50cddc7bd752'])

In [22]:
def get_all_install_ids_for_user(user):
  seconds_on_domain_per_session = get_collection_for_user(user, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_user(user, 'synced:interventions_active_for_domain_and_session')
  user_to_all_install_ids = get_user_to_all_install_ids()
  output = []
  output_set = set()
  if user in user_to_all_install_ids:
    for install_id in user_to_all_install_ids[user]:
      if install_id not in output_set:
        output_set.add(install_id)
        output.append(install_id)
  for item in seconds_on_domain_per_session:
    if 'install_id' not in item:
      continue
    install_id = item['install_id']
    if install_id not in output_set:
      output_set.add(install_id)
      output.append(install_id)
  for item in interventions_active_for_domain_and_session:
    if 'install_id' not in item:
      continue
    install_id = item['install_id']
    if install_id not in output_set:
      output_set.add(install_id)
      output.append(install_id)
  return output

def get_is_valid_user(user):
  install_ids = get_all_install_ids_for_user(user)
  if len(install_ids) != 1:
    return False
  return True

def get_valid_user_list():
  user_list = get_users_with_goal_frequency_set()
  output = []
  for user in user_list:
    if not get_is_valid_user(user):
      continue
    output.append(user)
  return output

#get_sessions_for_user('e0ea34c81d4b50cddc7bd752')
#valid_user_list = get_valid_user_list()
#print(len(valid_user_list))

1279


In [97]:
'''
function convert_date_to_epoch(date) {
  let start_of_epoch = moment().year(2016).month(0).date(1).hours(0).minutes(0).seconds(0).milliseconds(0)
  let year = parseInt(date.substr(0, 4))
  let month = parseInt(date.substr(4, 2)) - 1
  let day = parseInt(date.substr(6, 2))
  let date_moment = moment().year(year).month(month).date(day).hours(0).minutes(0).seconds(0).milliseconds(0)
  return date_moment.diff(start_of_epoch, 'days')
}

function convert_epoch_to_date(epoch) {
  let start_of_epoch = moment().year(2016).month(0).date(1).hours(0).minutes(0).seconds(0).milliseconds(0)
  start_of_epoch.add(epoch, 'days')
  return start_of_epoch.format('YYYYMMDD')
}

function timestamp_to_epoch(timestamp) {
  let start_of_epoch = moment().year(2016).month(0).date(1).hours(0).minutes(0).seconds(0).milliseconds(0)
  return moment(timestamp).diff(start_of_epoch, 'days')
}
'''

def convert_date_to_epoch(date):
  start_of_epoch = moment.now().timezone("US/Pacific").replace(years=2016, months=1, days=1, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  year = int(date[0:4])
  month = int(date[4:6])
  day = int(date[6:8])
  date_moment = moment.now().timezone("US/Pacific").replace(years=year, months=month, days=day, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  return date_moment.diff(start_of_epoch).days

def convert_epoch_to_date(epoch):
  start_of_epoch = moment.now().timezone("US/Pacific").replace(years=2016, months=1, days=1, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  start_of_epoch.add(days=epoch)
  return start_of_epoch.format('YYYYMMDD')

def timestamp_to_epoch(timestamp):
  start_of_epoch = moment.now().timezone("US/Pacific").replace(years=2016, months=1, days=1, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  return moment.unix(timestamp).timezone("US/Pacific").diff(start_of_epoch).days

#print(timestamp_to_epoch(1537059309631))
#print(convert_epoch_to_date(989))
#print(convert_date_to_epoch('20180915'))

In [42]:
def get_frequency_info_for_user(user):
  goal_frequencies = get_collection_for_user(user, 'synced:goal_frequencies')
  return goal_frequencies

#def get_frequency_info_for_goal_on_timestamp(user, goal, )

def get_sessions_for_user(user):
  seconds_on_domain_per_session = get_collection_for_user(user, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_user(user, 'synced:interventions_active_for_domain_and_session')
  print(seconds_on_domain_per_session[0])
  print(interventions_active_for_domain_and_session[0])
  output = []
  for item in seconds_on_domain_per_session:
    domain = item['key']
    session_id = item['key2']
    duration = item['val']

#print(get_sessions_for_user('c11e5f2d93f249b5083989b2'))
print(get_frequency_info_for_user('c11e5f2d93f249b5083989b2'))

[{'_id': ObjectId('5b669b07d206af0013ccf95c'), 'key': 'custom/spend_less_time_www.tumblr.com', 'val': '{"algorithm":"isoweek_alternating","onweeks":0,"timestamp":1533451014073}', 'synced': 0, 'timestamp': 1533451015932.0, 'userid': 'c11e5f2d93f249b5083989b2', 'install_id': 'e3a8a5a39bda2ef7c98a116b', 'collection': 'goal_frequencies', 'habitlab_version': '1.0.238', 'timestamp_local': 1533451014073.0}, {'_id': ObjectId('5b669b08d206af0013ccf95d'), 'key': 'reddit/spend_less_time', 'val': '{"algorithm":"isoweek_alternating","onweeks":0,"timestamp":1533451014059}', 'synced': 0, 'timestamp': 1533451016001.0, 'userid': 'c11e5f2d93f249b5083989b2', 'install_id': 'e3a8a5a39bda2ef7c98a116b', 'collection': 'goal_frequencies', 'habitlab_version': '1.0.238', 'timestamp_local': 1533451014059.0}, {'_id': ObjectId('5b669b08d206af0013ccf95e'), 'key': 'youtube/spend_less_time', 'val': '{"algorithm":"isoweek_alternating","onweeks":0,"timestamp":1533451014050}', 'synced': 0, 'timestamp': 1533451016056.0, '

In [24]:
#print(len(get_users_with_goal_frequency_set()))
print(valid_user_list[0])

c11e5f2d93f249b5083989b2


In [ ]:
def print_stats_on_install_records():
  user_to_all_install_ids = get_user_to_all_install_ids()
  users_with_goal_frequency_set = get_users_with_goal_frequency_set()
  users_with_missing_install_record = []
  users_with_zero_installs = []
  users_with_multiple_installs = []
  users_with_single_install = []

  for username in users_with_goal_frequency_set:
    if username not in user_to_all_install_ids:
      users_with_missing_install_record.append(username)
      continue
    install_ids = user_to_all_install_ids[username]
    if len(install_ids) == 0:
      users_with_zero_installs.append(username)
      continue
    if len(install_ids) > 1:
      users_with_multiple_installs.append(username)
      continue
    users_with_single_install.append(username)

  print('users with missing install record', len(users_with_missing_install_record))
  print('users with zero installs', len(users_with_zero_installs))
  print('users with multiple installs', len(users_with_multiple_installs))
  print('users with single install', len(users_with_single_install))


#print_stats_on_install_records()